In [1]:
import torch
import torchvision
import cv2
import numpy as np


In [2]:
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

model.load_state_dict(torch.load('best_model.pth'))

device = torch.device('cuda')
model = model.to(device)

In [3]:
import math
from time import sleep
dis_threshold = 0.1
def convert_angle(delta_x,delta_y):
    if(delta_x == 0):      #special case when x = 0 
        if(delta_y == 0):          #reach the destination
            return None
        elif(delta_y > 0):     
            return 90
        else:
            return -90
    temp = delta_y / delta_x
    
    if(delta_x > 0): #in case from -90 to 90 degree
        return math.degrees(math.atan(temp))
    else:
        
        if(delta_y > 0):  #case for 90 to 180 degree
            return 180 + math.degrees(math.atan(temp))
        else:       #case for -180 to -90 degree
            return math.degrees(math.atan(temp)) + 180 - 360
        
        
def update_info_straight(curr_x,curr_y,curr_angle, time):    #for keep space
    speed = 0.3      #currently defalut 1, can use calibration to change that
    curr_x = curr_x + math.cos(math.radians(curr_angle)) * time * speed
    curr_y = curr_y + math.sin(math.radians(curr_angle)) * time * speed

    return curr_x,curr_y

def reach_destination(curr_x, x, curr_y, y):  
    distance_apart = math.sqrt((curr_x - x)**2 + (curr_y - y)**2)       #use to calculate the distance from the destination to current lcoation
    
    if(distance_apart < dis_threshold): #within a range, then it mean it is close to the destination, then it return true.
        return True
    else:
        return False

def turn_left(leftspeed = 0, rightspeed = 0.6):    # use for easy control of the car
    robot.set_motors(leftspeed, rightspeed)

def turn_right(leftspeed = 0.6, rightspeed = 0):    
    robot.set_motors(leftspeed, rightspeed)

def spin_left(speed = 0.5):
    robot.left(speed)

def spin_right(speed = 0.5):
    robot.right(speed)

def straight(speed = 0.3):
    robot.forward(speed)

def check_angle(angle):     #use to convert the angle to be between -180 to 180 degree
    if(angle > 180):
        return -180 + angle - 180
    elif(angle < -180):
        return 180 + angle + 180
    else:
        return angle

In [4]:
mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [5]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)
display(widgets.HBox([image, blocked_slider]))

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [26]:
from jetbot import Robot

robot = Robot()
robot.stop()

curr_x = 0          #use to keep track on the x of the car current status
curr_y = 0          #use to keep track on the y of the car current status
curr_angle = 0      #use to keep track on the angle of the car current status, range from -180 to 180. 0 indicate straight
des_x = 0.5
des_y = 0.5
spin_around_time = 3.1


In [27]:
import torch.nn.functional as F
import time
import random


no_obj_go_time = 0.001
obj_spin_time_max = 0.1
obj_go_time = 0.5
go_forward_ct = 0
go_forward_before_redirect_max = 200 # #* 0.001s
start_time = time.time()
last_change = -1 # 0 is left, 1 is forward, 2 is right

def update(change):
    global blocked_slider, robot, curr_x, curr_y, curr_angle 
    global des_x, des_y, spin_time
    global go_forward_ct, start_time, last_change
    
    if not des_x or not des_y:
        print("hahaha")
    
    if reach_destination(curr_x, des_x, curr_y, des_y):
        #camera.unobserve(update, names='value')
        robot.stop()
        print("stop")
        camera.unobserve(update, names='value')
        robot.stop()
        
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    #print("current location", curr_x, curr_y, curr_angle )
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if last_change == 0:
        curr_angle = check_angle(curr_angle + (360 * (time.time()-start_time)/ spin_around_time) )  #make sure the angle is between -180 to 180 degree
    elif last_change == 1:
        curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, (time.time()-start_time)) 
    
    if prob_blocked < 0.5: #if no object
        target_angle = convert_angle(des_x - curr_x, des_y - curr_y)
        angle_diff = target_angle - curr_angle 
        
        #if go_forward_ct > random.randint(6,9) * 0.1 * go_forward_before_redirect_max and abs(angle_diff) > 2: 
        if abs(angle_diff) > 2:
            #go_forward_ct = 0
                #not pointing at the correct angle
                #adjust to the left while going straight if angle is negative and right if angle is positive
            #straight()
            #...
            if angle_diff < 0: # target < current
                spin_right()
            else:
                spin_left()

            time_to_turn = spin_around_time * abs(angle_diff) / 360     #calculate how long does the spin need to be
            #time_to_turn = 0.001
            sleep(time_to_turn)     #sleep to wait until the car turn enough
            robot.stop()
            
            curr_angle = target_angle    #x and y did not change, only the angle change to the target value in this case.
            last_change = -1
            print("current location", curr_x, curr_y, curr_angle )
            '''
            if angle_diff > 0 :
                curr_angle = check_angle(curr_angle + (360 * time_to_turn / spin_around_time) )
            else:
                curr_angle = check_angle(curr_angle - (360 * time_to_turn / spin_around_time) )
            '''
        else:
            
            #go straight 
            #go_forward_ct += 1
            straight()
            start_time = time.time()
            #sleep(no_obj_go_time)
            sleep(0.001)
            # curr_x, curr_y = update_info_straight(curr_x,curr_y,curr_angle, no_obj_go_time)   
            last_change = 1
            print("current location", curr_x, curr_y, curr_angle )

            
    else:
        #go_forward_ct = 0
        spin_left()
        #obj_spin_time = random.randint(1,9) * 0.1 * obj_spin_time_max
        start_time = time.time()
        obj_spin_time = 0.001
        sleep(obj_spin_time)
        #curr_angle = check_angle(curr_angle + (360 * obj_spin_time / spin_around_time) )  #make sure the angle is between -180 to 180 degree
        last_change = 0
        #straight()
        #sleep(obj_go_time)
        #curr_x, curr_y = update_info_straight(curr_x, curr_y,curr_angle,obj_go_time)
        print("current location", curr_x, curr_y, curr_angle )
        
    # time.sleep(0.001)
        
update({'new': camera.value})  # we call the function once to intialize

current location 0 0 45.0


In [28]:
camera.observe(update, names='value')  # this attaches the 'update' function to the 'value' traitlet of our camera

current location 0 0 45.0
current location 0.05685812168992457 0.056858121689924566 45.0
current location 0.09939811600372822 0.09939811600372822 45.0
current location 0.15150021872028946 0.15150021872028946 45.0
current location 0.19427674114793625 0.19427674114793625 45.0
current location 0.24446402824295055 0.24446402824295055 45.0
current location 0.2869495688284781 0.2869495688284781 45.0
current location 0.3331436949081763 0.33314369490817625 45.0
current location 0.38572846333388433 0.3857284633338842 45.0
current location 0.43467191263564514 0.43467191263564503 45.0
stop
current location 0.480284580793287 0.4802845807932869 45.0


In [29]:
camera.unobserve(update, names='value')
robot.stop()

ValueError: list.remove(x): x not in list

In [31]:
robot.stop()

In [5]:
def funcc(x, y):
    x = x + 1
    y = y + 1
    return x,y

In [6]:
a = 1
b = 1
a, b = funcc(a, b)

In [7]:
print(a)

2


In [10]:
import math
from time import sleep
def convert_angle(delta_x,delta_y):
    if(delta_x == 0):      #special case when x = 0 
        if(delta_y == 0):          #reach the destination
            return None
        elif(delta_y > 0):     
            return 90
        else:
            return -90
    temp = delta_y / delta_x
    
    if(delta_x > 0): #in case from -90 to 90 degree
        return math.degrees(math.atan(temp))
    else:
        
        if(delta_y > 0):  #case for 90 to 180 degree
            return 180 + math.degrees(math.atan(temp))
        else:       #case for -180 to -90 degree
            return math.degrees(math.atan(temp)) + 180 - 360
        
print(convert_angle(3 , 3))
print(convert_angle(3 , -3))
print(convert_angle(-3 , 3))
print(convert_angle(-3 , -3))

45.0
-45.0
135.0
-135.0


In [11]:
def update_info_straight(curr_x,curr_y,curr_angle, time):    #for keep space
    speed = 1       #currently defalut 1, can use calibration to change that
    curr_x = curr_x + math.cos(math.radians(curr_angle)) * time * speed
    curr_y = curr_y + math.sin(math.radians(curr_angle)) * time * speed

    return curr_x,curr_y

x, y = update_info_straight(3, 3, 45, 1.414)
print(x, y)
x, y = update_info_straight(3, 3, -45, 1.414)
print(x, y)
x, y = update_info_straight(3, 3, 135, 1.414)
print(x, y)
x, y = update_info_straight(3, 3, -135, 1.414)
print(x, y)

3.9998489885977784 3.999848988597778
3.9998489885977784 2.000151011402222
2.000151011402222 3.9998489885977784
2.000151011402222 2.0001510114022216
